In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL der Zielwebsite
url = "https://pokemondb.net/pokedex/all"

# Sende eine HTTP-GET-Anfrage
response = requests.get(url)

# HTML-Inhalt mit BeautifulSoup parsen
soup = BeautifulSoup(response.text, 'html.parser')

# Tabelle mit ID 'pokedex' finden
table = soup.find('table', {'id': 'pokedex'})

# Spaltennamen aus der Tabelle extrahieren
headers = [th.text.strip() for th in table.find('thead').find_all('th')]

# Zeilen der Tabelle extrahieren
rows = []
for tr in table.find('tbody').find_all('tr'):
    cells = [td.text.strip() for td in tr.find_all('td')]
    rows.append(cells)

# DataFrame mit Pandas erstellen
df = pd.DataFrame(rows, columns=headers)

# DataFrame als CSV speichern
csv_filename = "pokedex.csv"
df.to_csv(csv_filename, index=False)

print(f"DataFrame erfolgreich in {csv_filename} exportiert.")

DataFrame erfolgreich in pokedex.csv exportiert.


In [19]:
# CSV-Datei einlesen (ersetze 'pokemon.csv' mit deinem Dateipfad)
df = pd.read_csv('pokedex.csv')

# Mehrfache Typen splitten für bessere Analyse
df['Type'] = df['Type'].str.split()

# Hilfsfunktion, um den besten Eintrag je Typ zu finden
def find_best_by_stat(df, stat):
    best_per_type = {}
    for types in df['Type']:
        for t in types:
            type_df = df[df['Type'].apply(lambda x: t in x)]
            best_row = type_df.loc[type_df[stat].idxmax()]
            best_per_type[t] = best_row
    return best_per_type

# Stärkstes Pokémon pro Typ nach Total
total_best = find_best_by_stat(df, 'Total')

# Top 10 Angreifer insgesamt
top_10_attackers = df.nlargest(10, 'Attack')

# Durchschnittswerte pro Typ
average_stats = {}
for types in df['Type']:
    for t in types:
        type_df = df[df['Type'].apply(lambda x: t in x)]
        avg = type_df[['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].mean()
        average_stats[t] = avg

# Ergebnisse ausgeben
print("Stärkstes Pokémon pro Typ (nach Total):")
for t, row in total_best.items():
    print(f"{t}: {row['Name']} ({row['Total']} Punkte)")

print("\nTop 10 Angreifer insgesamt:")
print(top_10_attackers[['Name', 'Attack']])

print("\nDurchschnittswerte pro Typ:")
for t, avg in average_stats.items():
    print(f"{t}: {avg.to_dict()}")

Stärkstes Pokémon pro Typ (nach Total):
Grass: Sceptile Mega Sceptile (630 Punkte)
Poison: Eternatus Eternamax (1125 Punkte)
Fire: Groudon Primal Groudon (770 Punkte)
Flying: Rayquaza Mega Rayquaza (780 Punkte)
Dragon: Eternatus Eternamax (1125 Punkte)
Water: Kyogre Primal Kyogre (770 Punkte)
Bug: Pinsir Mega Pinsir (600 Punkte)
Normal: Arceus (720 Punkte)
Dark: Tyranitar Mega Tyranitar (700 Punkte)
Electric: Zekrom (680 Punkte)
Psychic: Mewtwo Mega Mewtwo X (780 Punkte)
Ground: Groudon Primal Groudon (770 Punkte)
Ice: Kyurem White Kyurem (700 Punkte)
Steel: Metagross Mega Metagross (700 Punkte)
Fairy: Diancie Mega Diancie (700 Punkte)
Fighting: Mewtwo Mega Mewtwo X (780 Punkte)
Rock: Tyranitar Mega Tyranitar (700 Punkte)
Ghost: Giratina Altered Forme (680 Punkte)

Top 10 Angreifer insgesamt:
                         Name  Attack
201      Mewtwo Mega Mewtwo X     190
274  Heracross Mega Heracross     185
956                   Kartana     181
473    Groudon Primal Groudon     180
475   